In [1]:
%load_ext autoreload
%autoreload 2
from build123d import *
from jupyter_cadquery import set_defaults, show, Camera, Collapse
import math
from functools import reduce, partial
from types import SimpleNamespace
from typing import TypeVar, Optional
from dataclasses import dataclass
from ipywidgets import interact
import ipywidgets as widgets
from pathlib import Path

set_defaults(
    viewer="Sidecar",
	edge_accuracy=0.0001,
	ticks=25,
	grid=[True, True, True],
	cad_width=780,
	axes=True,
	axes0=True,
	glass=True,
	orbit_control=True,
	reset_camera=Camera.CENTER,
	collapse=Collapse.LEAVES
)

Overwriting auto display for build123d BuildPart, BuildSketch, BuildLine, ShapeList


In [5]:
inside = Pos(0,0,HEIGHT / 2 + THICKNESS / 2) * Box(LENGTH, WIDTH, HEIGHT)

def __is_outer_edge(edge: Edge, y_bound: float, x_bound: float) -> bool:
    x, y, _ = edge.center().to_tuple()
    return (abs(y) == y_bound or abs(x) == x_bound)

def __is_inner_body_edge(edge: Edge, y_bound: float, x_bound: float, thickness: float) -> bool:
    x, y, z = edge.center().to_tuple()
    return -x_bound < x < x_bound and -y_bound <= y <= y_bound and round(z, 2) >= thickness 

def make_body(
        length: float,
        width: float,
        height: float,
        thickness: float,
        margin: float,
        lid_height: float,
        body_fillet: float,
        num_dividers: int,
        divider_thickness: float,
        chamfer_length: float,
        outer_bound: bool = True
    ):
    
    if outer_bound:
        outer_length, outer_width = length, width
        inner_length, inner_width = length - 2 * thickness, width - 2 * thickness
        lip_length, lip_width = length - thickness - margin / 2, width - thickness - margin / 2
        inner_height = height - 2 * thickness
        lip_height = height - thickness
    else:
        inner_length, inner_width = length, width
        outer_length, outer_width = length + 2 * thickness, width + 2 * thickness
        lip_length, lip_width = length + thickness - margin / 2, width + thickness - margin / 2
        inner_height = height
        lip_height = height + thickness
    
    base_profile = Rectangle(outer_length, outer_width)
    lip_bottom_profile = base_profile - Rectangle(inner_length, inner_width)
    lip_top_profile = base_profile - Rectangle(lip_length, lip_width)
    divider_profile = Rectangle(divider_thickness, inner_width + thickness / 2)

    obj = extrude(
        base_profile, thickness
    ) + extrude(
        Plane.XY.offset(thickness) * lip_bottom_profile, inner_height
    )
    
    if lid_height < thickness:
        obj = obj + extrude(
            Plane.XY.offset(lip_height) * lip_top_profile, lid_height
        )
    elif lid_height > thickness:
        obj = obj - extrude(
            Plane.XY.offset(lip_height).reverse() * lip_top_profile, lid_height
        )
    
    if body_fillet > 0:
        obj = fillet(obj.edges().filter_by(Axis.Z), body_fillet)
    
    if num_dividers > 0:
        divider_cutout = extrude(Plane.XY.offset(lip_height) * divider_profile, -(inner_height + thickness / 2))

        obj: Part = obj - reduce(lambda x, y: x + y, [
            loc * divider_cutout for loc in GridLocations(inner_length / (num_dividers + 1), 1, num_dividers, 1)
        ])

    if chamfer_length > 0:
        should_chamfer = partial(__is_outer_edge, x_bound=outer_length / 2, y_bound=outer_width / 2)
        obj = chamfer(ShapeList(filter(should_chamfer, obj.edges())), chamfer_length, chamfer_length)
        if num_dividers > 0:
            should_fillet = partial(__is_inner_body_edge, x_bound=inner_length / 2, y_bound=inner_width / 2, thickness=thickness)
            obj = fillet(ShapeList(filter(should_fillet, obj.edges().filter_by(Plane.YZ))), chamfer_length)
    
    return obj

def make_lid(
        length: float,
        width: float,
        thickness: float,
        margin: float,
        lid_height: float,
        body_fillet: float,
        chamfer_length: float,
        outer_bound: bool =True
    ):

    # Dimension correction

    if outer_bound:
        outer_length, outer_width = length, width
        lid_length, lid_width = length - thickness + margin / 2, width - thickness + margin / 2
    else:
        outer_length, outer_width = length + 2 * thickness, width + 2 * thickness
        lid_length, lid_width = length + thickness + margin / 2, width + thickness + margin / 2

    # Basic Shape

    base_profile = Rectangle(outer_length, outer_width)
    lid_top_profile = base_profile - Rectangle(lid_length, lid_width)
    obj = extrude(
        base_profile, thickness
    )

    # Lid correction

    if lid_height < thickness:
        obj = obj - extrude(
            Plane.XY.offset(thickness).reverse() * lid_top_profile, lid_height
        )
    elif lid_height > thickness:
        obj = obj + extrude(
            Plane.XY.offset(thickness) * lid_top_profile, lid_height
        )

    # Fillet body

    if body_fillet > 0:
        obj = fillet(obj.edges().filter_by(Axis.Z), body_fillet)

    # Chamfer edges

    if chamfer_length > 0:
        should_chamfer = partial(__is_outer_edge, x_bound=outer_length / 2, y_bound=outer_width / 2)
        obj = chamfer(ShapeList(filter(should_chamfer, obj.edges())), chamfer_length, chamfer_length)

    return obj

def make_divider(divider_thickness, inner_width, inner_height, thickness):
    return extrude(Rectangle(divider_thickness, inner_width), inner_height - thickness)

In [7]:
def assemble(
        length, width, height, thickness, margin, lid_height, body_fillet, divider_thickness, num_dividers, chamfer_length, printable_orientation=True
    ):
    lid_pos = Pos((0,0,height)) * Rot((0,180,0)) if not printable_orientation else Pos((0, width + margin))
    divider_pos = Pos((0,0,height - thickness)) * Rot((0,180,0)) if not printable_orientation else Pos((0, -width, divider_thickness / 2)) * Rot((0,90,0))
    return (
        make_body(
            length, width, height, thickness, margin, lid_height, body_fillet, num_dividers, divider_thickness, chamfer_length
        ),
        lid_pos * make_lid(length, width, thickness, margin, lid_height, body_fillet, chamfer_length),
        divider_pos * make_divider(divider_thickness, width - 2 * thickness, height - thickness, thickness)
    )

def show_assembly(length, width, height, thickness, margin, lid_height, body_fillet, divider_thickness, num_dividers, chamfer_length, printable_orientation):
    show(
        *assemble(length, width, height, thickness, margin, lid_height, body_fillet, divider_thickness, num_dividers, chamfer_length, printable_orientation),
        *assemble(226, 86, height, thickness, margin, lid_height, body_fillet, divider_thickness, num_dividers, chamfer_length, printable_orientation),
        names=["body", "lid", "divider"] + ["body", "lid", "divider"],
        ticks=30
    )

In [8]:
show_assembly(
    LENGTH,
    WIDTH,
    HEIGHT,
    THICKNESS,
    MARGIN, LID_HEIGHT, BODY_FILLET, DIVIDER_THICKNESS, NUM_DIVIDERS, LIP_CHAMFER, False)

++++++


In [9]:
LENGTH = 250
WIDTH = 100
HEIGHT = 32
THICKNESS = 4.2
LID_HEIGHT = 1.2
MARGIN = 0.21
BODY_FILLET = 2
DIVIDER_THICKNESS = 1.26
NUM_DIVIDERS = 11
LIP_CHAMFER = 1

In [65]:
body, lid, divider = assemble(
    LENGTH,
    WIDTH,
    HEIGHT,
    THICKNESS,
    MARGIN,
    LID_HEIGHT,
    BODY_FILLET,
    DIVIDER_THICKNESS,
    NUM_DIVIDERS,
    LIP_CHAMFER,
)
show(body, lid, divider)

++c


In [11]:
from utilities.export import export
from pathlib import Path

body, lid, divider = assemble(60, 60, 30, THICKNESS, MARGIN, LID_HEIGHT, BODY_FILLET, DIVIDER_THICKNESS, NUM_DIVIDERS, LIP_CHAMFER)

export(Path("outputs") / "Lockbox" / "60mmX60mmX30mm", body, lid, divider)

In [66]:
output_path : Path = Path("outputs") / "Lockbox" / f"{LENGTH}mmx{WIDTH}mmx{HEIGHT}mm"
output_path.mkdir(exist_ok=True)
export_stl(body, output_path / "body.stl")
export_stl(lid, output_path / "lid.stl")
export_stl(divider, output_path / "divider.stl")

True